In [228]:
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [229]:
train = pd.read_csv('/Users/taewan/Desktop/Project/predict_maker_cnn/train_data.csv')
test = pd.read_csv('/Users/taewan/Desktop/Project/predict_maker_cnn/test_data.csv')

In [230]:
train_list = []
train_label_list = []
for i in range(len(train)) :
    a = train['img_encode'][i].split(",")
    train_list.append(a)
    b = train['label'][i].split(",")
    train_label_list.append(b)

In [231]:
train_data = np.array(train_list, dtype = 'float32')
train_label = np.array(train_label_list, dtype = 'float32')

In [232]:
test_list = []
test_label_list = []
for i in range(len(test)) :
    a = test['img_encode'][i].split(",")
    test_list.append(a)
    b = test['label'][i].split(",")
    test_label_list.append(b)

In [233]:
test_data = np.array(test_list, dtype = 'float32')
test_label = np.array(test_label_list, dtype = 'float32')

In [234]:
def encode_label(label):
    if label == "nike":
        return [1,0]
    else:
        return [0,1]
    
train["label_encode"] = train.label.apply(encode_label)
test["label_encode"] = test.label.apply(encode_label)

train["img_encode"] = '[[' + train["img_encode"] + ']]'
test["img_encode"] = '[[' + test["img_encode"] + ']]'

In [277]:
tf.reset_default_graph()

np.random.seed(11111111)
tf.set_random_seed(11111111)

## first_layer

first_filter = 32

x = tf.placeholder(tf.float32, [None, 1024])
image_x = tf.reshape(x, [-1,32,32,1])

#print(image_x)
# 필터 크기 : 5x5x1, 필터 수 = 32 / tf.truncated_normal : 임의의 수 지정
# 스트라이드 = 1,1,1,1 / 패딩 : 특징 유실 방지(이미지 크기 보존)
first_conv_w = tf.Variable(tf.truncated_normal([5,5,1,first_filter],stddev = 0.1))
first_conv_h = tf.nn.conv2d(image_x, first_conv_w, strides=[1,1,1,1], padding = 'SAME')

#print(first_conv_w)
#print(first_conv_h)
first_conv_b = tf.Variable(tf.constant(0.1, shape = [first_filter]))
first_result = tf.nn.relu(first_conv_h + first_conv_b)

#print(first_conv_b)
#print(first_result)

first_pool = tf.nn.max_pool(first_result, ksize=[1,2,2,1],
                           strides = [1,2,2,1], padding = 'SAME')

#print(first_pool)

## second_layer

second_filter = 64

second_conv_w = tf.Variable(tf.truncated_normal([5,5,first_filter,
                                                 second_filter], stddev = 0.1))
#print(second_conv_w)
second_conv_h = tf.nn.conv2d(first_pool, second_conv_w,
                            strides = [1,1,1,1], padding = 'SAME')

second_conv_b = tf.Variable(tf.constant(0.1, shape =[second_filter]))
print(second_conv_b)
second_result = tf.nn.relu(second_conv_h + second_conv_b)

second_pool = tf.nn.max_pool(second_result, ksize = [1,2,2,1],
                            strides = [1,2,2,1], padding = 'SAME')

second_pool_flat = tf.reshape(second_pool, [-1, 8*8*second_filter])
print(second_pool_flat)

num_vec = 8*8*second_filter
num_neuron = 1024

w2 = tf.Variable(tf.truncated_normal([num_vec, num_neuron]))
b2 = tf.Variable(tf.Variable(tf.constant(0.1, shape = [num_neuron])))
hidden2 = tf.nn.relu(tf.matmul(second_pool_flat,w2) +b2)

# dropout : 일부러 일부 노드 연결 끊음 -> 오버피팅 방지
# hidden2 : 앞의 네트워크에서 전달된 값으로 keep_prob에 연결 비율 넣음

keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob)

w0 = tf.Variable(tf.zeros([num_neuron, 2]))
b0 = tf.Variable(tf.zeros([2]))
k = tf.matmul(hidden2_drop, w0) + b0
p = tf.nn.softmax(k)

#define loss (cost) function
t = tf.placeholder(tf.float32, [None, 2])
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=k,labels=t)) 
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# prepare session
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

<tf.Variable 'Variable_3:0' shape=(64,) dtype=float32_ref>
Tensor("Reshape_1:0", shape=(?, 4096), dtype=float32)


/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [278]:
#start training

i = 0
count = 0
k = 0

for _ in range(14):

    
    i += 1
    batch_xs, batch_ts = train_data[count:count+50], train_label[count:count+50]
    sess.run(train_step, feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})

    if i % 2 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            
            start = len(train_label) / 4 * c
           

            end = len(train_label) / 4 * (c+1)


            loss_val, acc_val = sess.run([loss, accuracy],

            feed_dict={x:train_data[int(start):int(end)], t:train_label[int(start):int(end)], keep_prob:1.0})

            loss_vals.append(loss_val)

            acc_vals.append(acc_val)

        loss_val = np.sum(loss_vals)

        acc_val = np.mean(acc_vals)

        print ('Step: %d, Loss: %f, Accuracy: %f'% (i, loss_val, acc_val))

    count += 50

#saver.save(sess, 'cnn_session')
sess.close()

Step: 2, Loss: 8.400848, Accuracy: 0.600267
Step: 4, Loss: 13.885121, Accuracy: 0.600267
Step: 6, Loss: 18.008326, Accuracy: 0.600267
Step: 8, Loss: 21.265877, Accuracy: 0.600267
Step: 10, Loss: 20.607372, Accuracy: 0.600267
Step: 12, Loss: 13.838503, Accuracy: 0.600267
Step: 14, Loss: 5.841381, Accuracy: 0.600267
